In [3]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup 
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.driving import count

import time
import serial

useSerial = True

if useSerial == True:
    # Estalish a connection with the CD48 board
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers


# Define a class of objects called Counter. Each counter (0-7) will get a widget checkbox 
# for Port A through Port D   
class Counter(object):
    def __init__(self, portA, portB, portC, portD):
        self.portA = portA
        self.portB = portB
        self.portC = portC
        self.portD = portD
    

# Set a variable for common height of widgets
widgetHeight = 20   


# Create Label Widgets for each Counter Row. Ex) the 0th row will say "Counter 0:"
counter_labels = []

for i in range(8):
    counter_labels.append(PreText(text='Counter: ' + str(i), width = 80, height = widgetHeight))
    
# define width of port labels
portLabelWidth = 30

# Create Label widgets for each Port Column
portA = PreText(text='A: ', width = portLabelWidth, height = widgetHeight)
portB = PreText(text='B: ', width = portLabelWidth, height = widgetHeight)
portC = PreText(text='C: ', width = portLabelWidth, height = widgetHeight)
portD = PreText(text='D: ', width = portLabelWidth, height = widgetHeight)


# create some blank widgets just to "take up space" so the layout looks nicer
blank = PreText(text='', width = 80, height = widgetHeight)
blank2 = PreText(text='', width = 10, height = widgetHeight)
blank3 = PreText(text='', width = 20, height = widgetHeight)

# Create Set Button to send counter information to CD48 board
set_button = Button(
    label='Set Counters',
    disabled=False,
    button_type='success' # 'success', 'info', 'warning', 'danger' or ''
)



# define width of checkbox widgets
CheckboxWidth = 30

# Create the list counters. Each element of counters is an object of the class Counter.
# This means each element of counters will have 4 widget checkboxes, each referred to as portA - portD
# Later, you can call the "activeness"of say counter 4, port D checkbox in this way: 
# counters[4].portD.active   
# If counters[4].portD.active = [0], then this box is checked. 
# If counters[4].portD.active = [], then this box is not checked.

counters = []

# loop through 8 times to create 4 widget checkboxes (portA, portB, portC, portD) 
# for each of counters 0 through 7
for i in range(8):
    counters.append(Counter(CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight)))
    

# create a list to store board settings in 
boardSettings = []

# the first element of boardSettings will just be the title "Board Settings:"
boardSettings.append(PreText(text="""Board Settings:""", width = 30, height = widgetHeight))

# for the remaining 12 items, just append blank ' ' which can be changed later
for i in np.arange(12):
    boardSettings.append(PreText(text=""" """, width = 30, height = widgetHeight))


# Define a function that can be called later. This function sends the 'P' command to the board, 
# then determines whether a given counter is currently "watching" a given port. 
# Then set the checkbox values accordingly (blah.active = [0] --> True, blah.active = [] --> False), 
# checkboxes are booleans
def set_checkboxes():
    
    # get the board settings
    s.write('P'.encode())
    time.sleep(0.5)
    settings = s.readlines()
    
    # add the board settings to the list boardSettings
    for i in np.arange(len(settings)):
        boardSettings[i+1].text = str(settings[i])
    
    # loop through 8 times to glean port settings for each counter
    for i in range(8):
        # for debugging purposes
        #print('counter = ', i)
        #print(settings[i][2:3])
        #print(settings[i][3:4])
        #print(settings[i][4:5])
        #print(settings[i][5:6])
        
        # the board writes back in bytes, not strings, hence the b'0' or b'1'.
        if settings[i][2:3] == b'0':
            counters[i].portA.active = []
        else:
            counters[i].portA.active = [0]
        
        if settings[i][3:4] == b'0':
            counters[i].portB.active = []
        else:
            counters[i].portB.active = [0]

        if settings[i][4:5] == b'0':
            counters[i].portC.active = []
        else:
            counters[i].portC.active = [0]

        if settings[i][5:6] == b'0':
            counters[i].portD.active = []
        else:
            counters[i].portD.active = [0]

# run the set_checkboxes function to give the checkboxes proper 
# intial values that reflect the current board settings
set_checkboxes()



# Create info labels that can warn the user if an error is made (you cannot clear a counter)
infoLabels = []

# append blank text ' ' that can be changed later
for i in range(8):
    infoLabels.append(PreText(text='', width = 400, height = widgetHeight))

    
# Create one horizontal box for each counter, which includes the port checkboxes
counterBoxes = []

for i in range(8):
    counterBoxes.append(row([counter_labels[i], 
                               counters[i].portA, 
                               counters[i].portB, 
                               counters[i].portC, 
                               counters[i].portD, 
                               blank2,
                               infoLabels[i]]))
    


# create a horizontal box with the port column labels
portsBox = row([blank, portA, portB, portC, portD])

# display our row with button and info_sent label
buttonRow = row([blank3, set_button])


# create a column of text widgets to display the board settings
boardSettingsColumn = column([boardSettings[0],
                              boardSettings[1],
                              boardSettings[2],
                              boardSettings[3],
                              boardSettings[4],
                              boardSettings[5],
                              boardSettings[6],
                              boardSettings[7],
                              boardSettings[8],
                              boardSettings[9],
                              boardSettings[10],
                              boardSettings[11],
                              boardSettings[12]])

# create the layout of our interface including all the rows and columns above
counterInterface = row([column([portsBox, 
                          counterBoxes[0], 
                          counterBoxes[1], 
                          counterBoxes[2], 
                          counterBoxes[3],
                          counterBoxes[4],
                          counterBoxes[5],
                          counterBoxes[6],
                          counterBoxes[7],
                          buttonRow]), boardSettingsColumn])


# define a function that will be run every time the Set Counters button is clicked
def send_info():
    
    # loop through 8 times, one for each counter
    for i in range(8):
        # set the values A-D to 0
        A = 0
        B = 0
        C = 0
        D = 0
        # clear the warning label
        infoLabels[i].text = ''
        # check the checkbox values of portA - portD, and change respective values as needed
        if counters[i].portA.active == [0]:
            A = 1
        if counters[i].portB.active == [0]:
            B = 1
        if counters[i].portC.active == [0]:
            C = 1
        if counters[i].portD.active == [0]:
            D = 1
            
        # if the sum of A - D is at least 1 (aka at least one port box is checked/True)
        # then write the counter setting command to the board
        if A + B + C + D > 0:
            countingCommand = 'S'+ str(i) + str(int(A)) + str(int(B)) + str(int(C)) + str(int(D)) + '\n'
            s.write(countingCommand.encode())
        # else (if the sum of A - D is 0), warn the user that this is not permitted. Do not send command to board
        else:
            infoLabels[i].text = 'Counter ' + str(i) + ' must watch at least 1 port.'
    
    # run the set_checkboxes function to ensure that the values of the checkboxes 
    # reflect the current board settings
    set_checkboxes()


    
# call the send_info function when the button is clicked.
set_button.on_click(send_info)



# This portion of the code is for the Three Graphs


useSerial = True

# Set up the CD48 board
#if useSerial:
#    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000) # May need to change the port number
    #s.write("S01100\n".encode()) 

    
    
# Initialize the data lists, time is the same for all three plots
# use a Column Data Source, intially all lists are empty

source = ColumnDataSource(dict(
    timeList=[], 
    singleCountsA=[], 
    singleCountsB=[], 
    coincidenceCountsAB=[]
    ))


# Create our 3 plots
singleCountsAPlot = figure(plot_height=200, plot_width=600, title="Single Counts on Port A",
              tools="crosshair,pan,reset,save,wheel_zoom")

singleCountsBPlot = figure(plot_height=200, plot_width=600, title="Single Counts on Port B",
              tools="crosshair,pan,reset,save,wheel_zoom")

coincidenceCountsABPlot = figure(plot_height=200, plot_width=600, title="Coincidence Counts on Ports A and B",
              tools="crosshair,pan,reset,save,wheel_zoom")


# Set up slider for user to choose counting time (100ms to 1000ms)
countingTimeSlider = Slider(start=100, end=1000, value=1000, step=10, title="Counting Time (ms)")
countingTime = countingTimeSlider.value # initialize the variable to store the counting interval

# How many points should be displayed on screen at a time. Can turn this into a user input widget
numberPointsDisplayed = 20


# Make the plots display only the most recent "numberPointsDisplayed" points
# This way, the graph will scroll to show the newest points
singleCountsAPlot.x_range.follow = "end"
singleCountsAPlot.x_range.follow_interval = numberPointsDisplayed*countingTime
    
singleCountsBPlot.x_range.follow = "end"
singleCountsBPlot.x_range.follow_interval = numberPointsDisplayed*countingTime
    
coincidenceCountsABPlot.x_range.follow = "end"
coincidenceCountsABPlot.x_range.follow_interval = numberPointsDisplayed*countingTime



# Make each point of each plot a circle
singleCountsAPlot.circle(x = 'timeList', y = 'singleCountsA', source = source, line_width=3, line_alpha=0.6)
singleCountsBPlot.circle(x = 'timeList', y = 'singleCountsB', source = source, line_width=3, line_alpha=0.6)
coincidenceCountsABPlot.circle(x = 'timeList', y = 'coincidenceCountsAB', source = source, 
                               line_width=3, line_alpha=0.6)

# Connect the points of each plot with a line
singleCountsAPlot.line(x = 'timeList', y = 'singleCountsA', source = source, line_width=2, line_alpha=0.6)
singleCountsBPlot.line(x = 'timeList', y = 'singleCountsB', source = source, line_width=2, line_alpha=0.6)
coincidenceCountsABPlot.line(x = 'timeList', y = 'coincidenceCountsAB', source = source, 
                               line_width=2, line_alpha=0.6)


# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1)

# Set up the reset button, to clear the graph of all data
resetGraphButton = Button(label="Reset")



# Set up time average value text box
#averageValueText = []

#for i in np.arange(3):
#    averageValueText.append(Paragraph(text="Plot " + str(i) + " Average: 0", width=400, height=40))

    

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0


# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    # retrieve the value of desired counting time
    countingTime = countingTimeSlider.value
    

    
    # use the global variable
    global runningTime
    
    # add the counting time to our running time
    runningTime = runningTime + countingTime
    
    
    
    if useSerial:
        # Request data from the CD48 counter
        s.write("c\n".encode())
        serialData = s.readline()
        counterData = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]
        
        new_data = dict(
            timeList=[runningTime],
            singleCountsA=[counterData[0]],
            singleCountsB=[counterData[1]],
            coincidenceCountsAB=[counterData[2]]
        )
    else:
        # append new data (the current time for timeList and random values for all others)
        new_data = dict(
            timeList=[runningTime],
            singleCountsA=[np.random.randint(0, countingTime)],
            singleCountsB=[np.random.randint(0, countingTime)],
            coincidenceCountsAB=[np.random.randint(0, countingTime)]
        )
    
    # Stream the new data to the plots.
    source.stream(new_data)
    
    '''
    # Append the most recently received count to list n
    if useSerial:
        # Request data from the CD48 counter
        s.write("c\n".encode())
        serialData = s.readline()
        data = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]
        singleCountsA.append(data[0])
        singleCountsB.append(data[1])
        coincidenceCountsAB.append(data[2])
    else:
        # generate random numbers from 0 to dt (in miliseconds)
        singleCountsA.append(np.random.randint(0, countingTime))
        singleCountsB.append(np.random.randint(0, countingTime))
        coincidenceCountsAB.append(np.random.randint(0, countingTime))
        
    '''
    
    
    # Update the average count in the text box
    #averageValueText[0].text = "Plot 1 Average Counts: %d " % (np.mean(singleCountsA))
    #averageValueText[1].text = "Plot 2 Average Counts: %d " % (np.mean(singleCountsB))
    #averageValueText[2].text = "Plot 3 Average Counts: %d " % (np.mean(coincidenceCountsAB))
    
    
    



# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        #global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime)
    
    # else if the STOP button is on
    elif startStopButtonGroup.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        

# Define a function to be called when the reset button is pressed.   
def reset_plot():
    
    # set the running time back to 0
    global runningTime
    runningTime = 0
    
    # empty all data lists in source
    source.data = {k: [] for k in source.data}
    

def change_callback_period(attr, old, new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        #global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime)
        
        
    

    
    
# When the START/STOP button is clicked, call the change_periodic_callback function
startStopButtonGroup.on_click(change_periodic_callback)


# When the reset button is clicked, call the reset_plot function
resetGraphButton.on_click(reset_plot)

countingTimeSlider.on_change('value', change_callback_period)

# Set up layouts and add to document

# create a column of user input widgets
userInputWidgets = column(startStopButtonGroup, 
                          countingTimeSlider, 
                          resetGraphButton#, 
                          #averageValueText[0],
                          #averageValueText[1],
                          #averageValueText[2]
                         )


# create a layout with the set counters interface from the first half of this code
setCountersLayout = layout([[counterInterface]], sizing_mode='scale_both')

# create a column with our three plots
plotsColumn = column([singleCountsAPlot, 
                      singleCountsBPlot, 
                      coincidenceCountsABPlot
                     ])

# create a layout for the graphs interface, with the input widgets and the three plots
displayCountsLayout = layout([[row([userInputWidgets, plotsColumn])]] , sizing_mode='scale_both')

# create a tab for the set counters interface
setCountersTab = Panel(child = setCountersLayout, 
                       title="Set Counters")

# create a tab for the graphs interface
displayCountsTab = Panel(child = displayCountsLayout, 
                         title="Graphs")

# combine all tabs into one layout
currentDocumentTabs = Tabs(tabs=[ setCountersTab, displayCountsTab ])


# Add tabs to current document
curdoc().add_root(currentDocumentTabs)

curdoc().title = "Coincidence Counting Interface"